In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
appRecord = pd.read_csv("application_record.csv");
creditRecord = pd.read_csv("credit_record.csv");

In [3]:
appRecord.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [4]:
creditRecord.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [5]:
creditRecord.insert(1,"STATUSINT",creditRecord['STATUS']);

In [6]:
creditRecord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   STATUSINT       1048575 non-null  object
 2   MONTHS_BALANCE  1048575 non-null  int64 
 3   STATUS          1048575 non-null  object
dtypes: int64(2), object(2)
memory usage: 32.0+ MB


In [7]:
"""
Original Data
0: 1-29 days past due 
1: 30-59 days past due 
2: 60-89 days overdue 
3: 90-119 days overdue 
4: 120-149 days overdue 
5: Overdue or bad debts, write-offs for more than 150 days 
C: paid off that month 
X: No loan for the month
-------------------------
Numerical Encoding
1: 1-29 days past due 
2: 30-59 days past due 
3: 60-89 days overdue 
4: 90-119 days overdue 
5: 120-149 days overdue 
6: Overdue or bad debts, write-offs for more than 150 days 
0: paid off that month 
0: No loan for the month
"""
def statusToInt(status):
    if status == "X": return 0
    elif status == "C": return 0
    elif status == "0": return 1
    elif status == "1": return 2
    elif status == "2": return 3
    elif status == "3": return 4
    elif status == "4": return 5
    else: return 6

In [8]:
creditRecord["STATUSINT"]=creditRecord["STATUS"].map(statusToInt)

In [9]:
print(creditRecord)

              ID  STATUSINT  MONTHS_BALANCE STATUS
0        5001711          0               0      X
1        5001711          1              -1      0
2        5001711          1              -2      0
3        5001711          1              -3      0
4        5001712          0               0      C
...          ...        ...             ...    ...
1048570  5150487          0             -25      C
1048571  5150487          0             -26      C
1048572  5150487          0             -27      C
1048573  5150487          0             -28      C
1048574  5150487          0             -29      C

[1048575 rows x 4 columns]


In [10]:
creditRecord.insert(1,"TOTALMONTH",creditRecord['MONTHS_BALANCE']);

In [11]:
creditRecord["TOTALMONTH"]=creditRecord["MONTHS_BALANCE"].map(lambda x: -1*x);

In [12]:
maximum = creditRecord.groupby("ID").max()

In [13]:
average = creditRecord.groupby("ID").sum()

In [14]:
maximum.head()

,TOTALMONTH,STATUSINT,MONTHS_BALANCE,STATUS
ID,,,,
5001711,3,1,0,X
5001712,18,1,0,C
5001713,21,0,0,X
5001714,14,0,0,X
5001715,59,0,0,X


In [15]:
average.head()

,TOTALMONTH,STATUSINT,MONTHS_BALANCE
ID,,,
5001711,1.5,0.750000,-1.5
5001712,9.0,0.526316,-9.0
5001713,10.5,0.000000,-10.5
5001714,7.0,0.000000,-7.0
5001715,29.5,0.000000,-29.5


In [16]:
del maximum['STATUSINT']
del maximum['MONTHS_BALANCE']
del maximum['STATUS']
maximum.head()

,TOTALMONTH
ID,
5001711,3
5001712,18
5001713,21
5001714,14
5001715,59


In [19]:
#del average['TOTALMONTH']
#del average['MONTHS_BALANCE']
average.head()

,STATUSINT
ID,
5001711,0.750000
5001712,0.526316
5001713,0.000000
5001714,0.000000
5001715,0.000000


In [25]:
# This is two custom features. TOTALMONTH is the duration of the record and STATUSINT is the average of STATUS after 
# the numerical encoding of 
"""
1: 1-29 days past due 
2: 30-59 days past due 
3: 60-89 days overdue 
4: 90-119 days overdue 
5: 120-149 days overdue 
6: Overdue or bad debts, write-offs for more than 150 days 
0: paid off that month 
0: No loan for the month
"""
customData = data = pd.merge(
    maximum,
    average,
    on="ID",
    how="inner"
)
customData.head()

,TOTALMONTH,STATUSINT
ID,,
5001711,3,0.750000
5001712,18,0.526316
5001713,21,0.000000
5001714,14,0.000000
5001715,59,0.000000


In [26]:
data = pd.merge(
    appRecord,
    customData,
    on="ID",
    how="inner"
)

In [27]:
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH,STATUSINT
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15,0.187500
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14,0.200000
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29,0.233333
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4,0.400000
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26,0.000000


In [28]:
data.info

<bound method DataFrame.info of             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0      5008804           M            Y               Y             0   
1      5008805           M            Y               Y             0   
2      5008806           M            Y               Y             0   
3      5008808           F            N               Y             0   
4      5008809           F            N               Y             0   
...        ...         ...          ...             ...           ...   
36452  5149828           M            Y               Y             0   
36453  5149834           F            N               Y             0   
36454  5149838           F            N               Y             0   
36455  5150049           F            N               Y             0   
36456  5150337           M            N               Y             0   

       AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          

In [34]:
# Everything before the last column and the first column is a feature for a crude model. 
X = data.iloc[:data.size,1:19];
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26


In [35]:
# The last column STATUSINT is what we are trying to predict. 
y = data.iloc[:data.size,19:20];
y.head()

,STATUSINT
0,0.187500
1,0.200000
2,0.233333
3,0.400000
4,0.000000


In [53]:
#Let's transform each feature into numerical features using default encoders. 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label_encoder = LabelEncoder();
#X['CODE_GENDER'] = label_encoder.fit_transform(X['CODE_GENDER']);
#X['FLAG_OWN_CAR'] = label_encoder.fit_transform(X['FLAG_OWN_CAR']);
#X['FLAG_OWN_REALTY'] = label_encoder.fit_transform(X['FLAG_OWN_REALTY']);
#X['NAME_INCOME_TYPE'] = label_encoder.fit_transform(X['NAME_INCOME_TYPE']);
#X['NAME_EDUCATION_TYPE'] = label_encoder.fit_transform(X['NAME_EDUCATION_TYPE']);
#X['NAME_FAMILY_STATUS'] = label_encoder.fit_transform(X['NAME_FAMILY_STATUS']);
#X['NAME_HOUSING_TYPE'] = label_encoder.fit_transform(X['NAME_HOUSING_TYPE']);
"""
Some people don't have occupation information. This will throw a TypeError for the label_encoder. Let's fix this. 
"""
def jobEncodingPrep(status):
    #print(status)
    if pd.isna(status) : return "No Job Info"
    else: return status
#X["OCCUPATION_TYPE"]=X["OCCUPATION_TYPE"].map(jobEncodingPrep)
#print(X)

# This is a dumb encoding, as occupation should be one-hot encoded. 
X['OCCUPATION_TYPE'] = label_encoder.fit_transform(X['OCCUPATION_TYPE']);

In [54]:
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH
0,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,12,2.0,15
1,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,12,2.0,14
2,1,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,17,2.0,29
3,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,15,1.0,4
4,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,15,1.0,26


In [65]:
from sklearn.ensemble import RandomForestRegressor
# as a test, make a forest model with just a single feature. 
clf = RandomForestRegressor();
clf.fit(X, y)

C:\Users\yun91\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor()

In [69]:
# see the first tree within the forest
print(clf.estimators_[0])

DecisionTreeRegressor(max_features='auto', random_state=2133953692)


In [70]:
# export as dot file
# on the command line, type dot tree.dot -Tpng -o tree.png to conver into png. 
from sklearn.tree import export_graphviz;
import os;
localFilePath = "C:/Users/yun91/Documents/GitHub/vis/data"
export_graphviz(
    clf.estimators_[0],
    out_file = os.path.join(localFilePath,"tree.dot"),
    rounded = True,
    filled = True)  

In [68]:
print(clf.score(X,y));

0.8371136851669764
